# Enzyme Kinetics Four Ways

This is an example of the kind of math you will need to do for the assignment.

## IMPORTANT! Go to the end of this notebook and execute the final code block FIRST. 

The last block of code contains the definitions for the functions used in this document.



In [ ]:
SN = 329644     ## Which data set should I use?
print(f"I will use data set: v{SN % 7}")

##########################################################
### Create Data set conc (mM), delta AU
##########################################################

data = [[0.05, 0.010],
        [0.1,  0.010],
        [0.25, 0.021],
        [0.5,  0.042],
        [1.0,  0.067],
        [2.5,  0.105],
        [5.0,  0.160],
        [8.0,  0.171],
        [20.0, 0.232],
        [30.0, 0.241]]

##########################################################
### Calculate the rates of reaction from the data.
##########################################################

data = np.array(data)   ### Convert the above list of lists to a 2D array.

### Collect concentrations and amount reacted data as two separate arrays

conc = data[:,0]       ### mM     - all rows, column 0
rate = data[:,1]       ### AU / min    - all rows, column 1



## Michaelis-Menten Plot

In [ ]:
##########################################################
### Use the MM_Plot function to curve fit the data, get 
### Vmax and KM and plot the data and the fitted curve.
##########################################################

x = conc
y = rate

answer = MM_Plot(x, y)        ### Call MM_Plot function
                              ### It will perform the curve fit and return a list of values (Vmax and KM)

V_max, KM = answer            ### Extract the values from the list

##########################################################
### Print the Vmax and KM values. NOTE: They are Uncertainties
### values, not floating point. Use unp.nominal_values(x) to extract
### the floating point value for x.  Use unp.std_devs(x) to extract
### the error component of x.
##########################################################

print(f"The Vmax is {V_max:0.3f}")
print(f"The KM is {KM:0.3f}")
print(unp.nominal_values(V_max))
print(unp.std_devs(V_max))


## Lineweaver Burke Plot

In [ ]:
##########################################################
### Use the MM_Plot function to curve fit the data, get 
### Vmax and KM and plot the data and the fitted curve.
##########################################################

x = 1/conc
y = 1/rate

##########################################################
### Call Linear_Plot function
### It will perform the linear fit and return a list of values (slope and intercept)
### You cal also change the labels of the plot
##########################################################

answer = Linear_Plot(x, y, title ="Enter Title Here", xaxis = "x-axis", yaxis = "y-axis")        

slope, intercept = answer     ### Extract the values from the list

V_max  = 1 / intercept        ### calculate MM parameters frol slope and intercept
KM = V_max * slope           

##########################################################
### Print the Vmax and KM values. NOTE: They are Uncertainties
### values, not floating point. Use unp.nominal_values(x) to extract
### the floating point value for x.  Use unp.std_devs(x) to extract
### the error component of x.
##########################################################

print(f"The Vmax is {V_max:0.3f}")
print(f"The KM is {KM:0.3f}")


## Eadie-Hofstee Plot

In [ ]:
##########################################################
### Steal the code above and use it to obtain KM and Vmax 
### using the Eadie-Hofstee plot
##########################################################


## Hanes-Woolf Plot

In [ ]:
##########################################################
### Steal the code above and use it to obtain KM and Vmax 
### using the Hanes-Woolf plot
##########################################################


## RUN THIS FIRST

Execute the final bock in this notebook below to establish all the functions and tools required in the code above.

In [45]:
from scipy.optimize import curve_fit
import numpy as np                       ## import the tools of NumPy but use a shorter name
from matplotlib import pyplot as plt

####################################
### Install UNCERTAINTIES package 
####################################

!pip install uncertainties  # to install in Colab. Add it back to avoid installing again and again.

####################################
### Import versions of NumPy and Math that use uncertainty 
####################################

import uncertainties as un
from uncertainties import unumpy as unp
from uncertainties import umath as um




def MM_Plot(x, y):

    ####################################
    ### Define the MM equation for the curve fit as a function
    ####################################

    def MMplot(S, Vmax, KM):
        v = Vmax * S / (S + KM)
        return(v)
        
    ####################################
    ### Perform the curve fit
    ####################################
    
    params, stats = curve_fit(MMplot, x, y)   ## two objects are returned
    
    ####################################
    ### Interpret the results
    ####################################
    
    v_max, KM = params   ### pull out the two values in the params object
    
    perr = np.sqrt(np.diag(stats))        ### convert covariance matrix to stdev values
    stdev_v_max, stdev_KM = perr          ### pull out the two stdev values 
    
    v_max_u = un.ufloat(v_max,stdev_v_max)  ## create a value with uncertainty built in
    KM_u = un.ufloat(KM,stdev_KM)      
    
    ################################
    ### make a list of x values from zero to the end of the line
    ################################
    
    x_fit = np.linspace(0, np.max(x), 100) 
    
    ################################
    ### Feed that list into the function for the line fit
    ################################
    
    y_fit = MMplot(x_fit,v_max,KM)
    
    ######################
    ### Create an empty plot
    #####################
    
    plt.rcdefaults()           
    plt.figure(figsize=(5,4))  
    
    ######################
    ### Plot the data and the curve fit line
    #####################
    
    plt.plot(x, y, "ko")
    plt.plot(x_fit, y_fit, "k-")   
                                                                                                     
    ######################
    ### Add some style
    #####################
    
    plt.ylim(0, None)
    plt.xlim(0, None)
    
    plt.ylabel(r"$\nu$")  
    plt.xlabel(r"$[S]$")
    plt.title("Michaelis-Menten Plot")
    
    ######################
    ### Display and export the plot
    #####################
    
    plt.savefig("MM_plot.pdf")
    plt.show()

    return v_max_u, KM_u

def Linear_Plot(x, y, title ="Enter Title Here", xaxis = "x-axis", yaxis = "y-axis"):

    def linear(x, slope, intercept):  ### Take x values, slope and intercept and return the y values
        y = slope * x + intercept
        return(y)

    params, stats = curve_fit(linear, x, y)   ## two objects are returned
    
    ####################################
    ### Interpret the results
    ####################################
    
    slope, intercept = params   ### pull out the two values in the params object
    
    perr = np.sqrt(np.diag(stats))        ### convert covariance matrix to stdev values
    stdev_slope, stdev_intercept = perr   ### pull out the two stdev values 
    
    slope_u = un.ufloat(slope,stdev_slope)               ## create a value with uncertainty built in
    intercept_u = un.ufloat(intercept,stdev_intercept)  
    
        
    ################################
    ### make a list of x values from zero to the end of the line
    ################################
    
    x_fit = np.linspace(0, np.max(x), 100)  ## 100 points from 0 to the highest value on LB plot x-axis
    
    ################################
    ### Feed that list into the function for the line fit
    ################################
    
    y_fit = linear(x_fit, slope, intercept)
    
    ######################
    ### Create an empty plot
    #####################
    
    plt.rcdefaults()           
    plt.figure(figsize=(5,4))  
    
    ######################
    ### Plot the data and the curve fit line
    #####################
    
    plt.plot(x, y, "ko")
    plt.plot(x_fit, y_fit, "k-")   

    ######################
    ### Add some style
    #####################
    
    plt.ylim(0, None)
    plt.xlim(0, None)
    
    plt.ylabel(yaxis)
    plt.xlabel(xaxis)
    plt.title(title)
    
    ######################
    ### Display and export the plot
    #####################
    
    plt.savefig("LB_plot_err.pdf")
    plt.show()

    return slope_u, intercept_u
